## Sentinel-2 Salt Marsh Mapping over Morecambe Bay
Author: Olivia Brennan\
Institution: University College London\
Date: 27/08/2025

### 1. Preliminary setup:

In [2]:
#import libraries 

import ee 
import geemap
import datetime
import folium
from folium import plugins
import numpy as np

# initialize GEE
ee.Authenticate() 
ee.Initialize(project='x-avenue-463113-j8') #project ID for 'Olivia's Dissertation project'

In [3]:
# Define the upper boudnary of salt marsh used to clip the scenes.
# based on HAT shoreline
boundary = geemap.shp_to_ee(r"C:\Users\oabre\OneDrive - University College London\olivia ucl\DISSERTATION\PYTHON\ShorelineExtraction\clean_shoreline_shapefiles\HT_20210527_10m.shp")

#define visualisations for RGB & Normalized Difference Vegetation Index 
RGB_vis = {
    'bands': ['B4', 'B3', 'B2'],  # True color
    'min': 0,
    'max': 2000,  
    'gamma': [1.2, 1.2, 1.2]  
}

ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['black', 'white', 'green']
}

masked_ndvi_vis = ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'green']
}

### 2. Call and clip a sentinel scene based on HAT. Apply NDVI thresholding.
Can use either a date in winter or a date in summer.\
Compute NDVI based on spectral bands 4 and 8.\
Extract vegetation based on NDVI>0.2 threshold.\
Toggle layers on and off using the interactive map

In [5]:
#summer date:
#start_date = '2018-07-19'
#end_date = '2018-07-20'

#winter date:
start_date = '2019-01-28'
end_date = '2019-01-29'

#call the set of scenes
sentinel_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                       .filterDate(start_date, end_date) 
                       .filterBounds(boundary)
                       .median())
                       
sentinel_image = sentinel_collection.clip(boundary) #clip to ROI

# Compute NDVI 
ndvi = sentinel_collection.normalizedDifference(['B8', 'B4']).rename('NDVI').clip(boundary)

# Create binary mask for vegetation (NDVI > 0.2)
veg_mask = ndvi.gt(0.2).selfMask()  # selfMask removes zeros

# Apply mask to NDVI
masked_ndvi = ndvi.updateMask(veg_mask)

#display the map:
Map = geemap.Map(center=(54.1, -2.8), zoom=10) #load map centred on ROI
Map.addLayer(sentinel_image, RGB_vis, 'Sentinel-2 RGB') #the true colour image
Map.addLayer(ndvi, ndvi_vis, 'NDVI') 
Map.addLayer(masked_ndvi, masked_ndvi_vis, 'masked_NDVI') 
Map

Map(center=[54.1, -2.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

### 3. Extract & export vegetation to polygon
Results will be exported to Google Drive

In [9]:
#multiply by 1000 for formatting
int_ndvi = masked_ndvi.multiply(1000).toInt()

#convert to vector
vectors = int_ndvi.reduceToVectors(
    geometry=boundary.geometry().bounds(),  # Simplify geometry
    scale=30,  
    geometryType='polygon',
    eightConnected=False,
    maxPixels=1e10
)

In [10]:
#Run the GEE export task:
task = ee.batch.Export.table.toDrive(
    collection=vectors,
    description=f'NDVI_Vegetation_Boundary30{start_date}',
    folder='GEE_exports',
    fileFormat='SHP'
)
task.start()